# Manipulating Norm Vectors

We use Doc2Vec to create sentence representations over contracts.
Using such representations, we get a random norm sentence representation and modal verb (shall, may, must, etc) representations.
We subtract the norm representation by the modal verb representation in the norm.
Then, we add a modal verb representation to the norm with an oposite deontic meaning (e.g., remove a prohibition and add a permission).
As result, we expect to obtain potential conflicts by 

## Doc2Vec

We train a doc2vec model using norm sentences from 169 manufacturing contracts.


In [1]:
import sys
import pickle
import numpy as np
import process_norms
from process_norms import ProcessNorms

Using TensorFlow backend.
/home/aires/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
d2v_path = 'models/model_2017-11-03_21-39-30.doc2vec'
dict_path = 'models/model_2017-11-03_21-39-30.pkl'
sent_cls_path = 'sentence_classifier/classifiers/17-11-03_18:45/sentence_classifier_17-11-03_18:45.pkl'
sent_cls_names_path = 'sentence_classifier/classifiers/17-11-03_18:45/sentence_classifier_dict_17-11-03_18:45.pkl'

In [3]:
def load_dict(path):

    dict_file = open(path, 'r')
    p_dict = pickle.load(dict_file)

    return p_dict

In [4]:
def print_list(l):
    
    return ' '.join(l)

In [5]:
def print_similar(similar_vecs):
# Run over similar vectors and print the sentences.
    for similar in similar_vecs:
        print '{}\t {}\n'.format(print_list(sent_dict[similar[0]]), similar[1])

In [6]:
# Instantiate the ProcessNorms class.
pn = ProcessNorms(d2v_path, sent_cls_path, sent_cls_names_path)

In [7]:
# Load dict.
sent_dict = load_dict(dict_path)

In [15]:
# Get a random norm sentence.
random_key = 'SENT_399'
if random_key in pn.doc2vec_model.docvecs:
    rand_sent_vec = pn.doc2vec_model.docvecs[random_key]
    print print_list(sent_dict[random_key])
else:
    print "Unknown key."

24 25 (B) Before making an offer of employment to any individual who is employed by Motorola during the term of this Agreement, ASE shall provide notice of its intent to make such an offer to Motorola's designated manager, and ASE and Motorola shall then discuss the circumstances which led to ASE's potential employment offer and the impact on their business relationship should any offer be made by ASE and accepted by such individual.


In [16]:
# Subtract the obligation word from the sentence.
sub_vector = pn.doc2vec_model['shall']  # Get subtraction vector.
add_vector = pn.doc2vec_model['may']  # Get adition vector.
no_modal = np.subtract(rand_sent_vec, sub_vector)  # Subtract vector from our sentence representation.
new_sent = np.add(no_modal, add_vector)  # Add vector to the sentence representation.

In [17]:
# Find similar vectors based on our new vector.
similar_vecs = pn.doc2vec_model.docvecs.most_similar([new_sent], topn=3)
print_similar(similar_vecs)

(B) Before making an offer of employment to any individual who is employed by Motorola during the term of this Agreement, ASE shall provide notice of its intent to make such an offer to Motorola's designated manager, and ASE and Motorola shall then discuss the circumstances which led to ASE's potential employment offer and the impact on their business relationship should any offer be made by ASE and accepted by such individual.	 0.42848226428

An interim franchise must be in writing and must make certain disclosures, including that it is an interim franchise and that the franchisor has the right not to renew the franchise at the end of the term based upon a lawful determination to withdraw from marketing activities in the geographic area in which the franchisee operates.	 0.415696650743

The parties will not disclose to any third party that notice has been given, that discussions are taking place or the content of any such notice or discussions, except that either party may issue any p

In [11]:
# Find similar vectors based on the original sentence.
similar_vecs = pn.doc2vec_model.docvecs.most_similar(random_key, topn=3)
print_similar(similar_vecs)

All notices shall be sent by registered or certified mail, return receipt requested, by delivery or by Federal Express or other comparable courier providing proof of delivery, and shall be deemed duly given and received (i) if mailed, on the tenth business day following the mailing thereof, or (ii) if sent by courier or if delivered, the date of its receipt (or, if such day is not a business day, the next succeeding business day).	 0.551526606083

If order acknowledgment is not received with [*], the WA will be deemed to be accepted by Supplier.	 0.546704411507

Each Firm Purchase Order shall specify the quantity or, if more than one shipment is requested, quantities of Finished Product ordered, the requested delivery date or dates, the delivery address(es) and any applicable shipping information (if different from that specified in Section 2.3).	 0.531639814377

